First we are gonna read all the sample and to extract the shortest length

In [2]:
!pip install matplotlib
!pip install giotto-tda
!pip install numpy
!pip install scipy
!pip install ipykernel
!pip install --upgrade nbformat

!pip install dtaidistance
!pip install pandas openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached giotto-tda-0.1.4.tar.gz (99 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Packages installed from PyPI cannot depend on packages which are not also hosted on PyPI.
giotto-tda depends on pycairo@ https://storage.googleapis.com/l2f-open-models/giotto-learn/windows-binaries/pycairo/pycairo-1.18.2-cp313-cp313m-win_amd64.whl



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from scipy.io import wavfile
import os
import matplotlib.pyplot as plt
import numpy as np


directory = './vowels/Control/A'

shortestLength = 9999
allSampleRate = -1
allSameSampleRate = True

actualStartOfFilename = ''
aggregate = True
aggregatedVoices = []
for filename in os.listdir(directory):
    startOfFilename = filename.split('a')[0]
    if(actualStartOfFilename != startOfFilename):
        aggregate = False
        actualStartOfFilename = startOfFilename
    f = os.path.join(directory, filename)
    samplerate, data = wavfile.read(f)
    length = data.shape[0] / samplerate
    allSampleRate = samplerate if allSampleRate == -1 else samplerate
    allSameSampleRate = samplerate == allSampleRate
    shortestLength = length if shortestLength > length else shortestLength
    
    

print(f"All the audios have the same sample rate: {allSameSampleRate}")
print(f"The sample rate is {allSampleRate} data per second")
print(f"The shortest audio length is {shortestLength}")
print(f"The number of data per audio to analyse is {shortestLength*samplerate}")



All the audios have the same sample rate: True
The sample rate is 44100 data per second
The shortest audio length is 0.677641723356009
The number of data per audio to analyse is 29884.0


In [ ]:
from dtaidistance import dtw_barycenter
# We aggregate the three audio files. As an idea it would be to short the signal between the three audios instead of in general
actualStartOfFilename = ''
aggregate = True
aggregatedVoices = []
voicesToAggregate = []
numberOfFilesMixed = 0
for filename in os.listdir(directory):
    startOfFilename = filename.split('a')[0]
    if(actualStartOfFilename != startOfFilename):
        aggregate = False
       
    f = os.path.join(directory, filename)
    samplerate, data = wavfile.read(f)
    if(aggregate):
        numberOfFilesMixed += 1
        voicesToAggregate.append(data.astype(np.float64))
        # aggregatedVoices[len(aggregatedVoices)-1] += data[:int(shortestLength*samplerate)] # To do simple mean

    else:
        if(len(voicesToAggregate) > 0):
            #  aggregatedVoices[len(aggregatedVoices)-1]  =  (actualStartOfFilename, aggregatedVoices[len(aggregatedVoices)-1] / numberOfFilesMixed) # For simple mean
            aggregatedVoice = dtw_barycenter.dba(voicesToAggregate,voicesToAggregate[0], use_c=True)
            aggregatedVoices.append(aggregatedVoice[:int(shortestLength*samplerate)])
            
        numberOfFilesMixed = 1
        aggregate=True
        voicesToAggregate = []
        voicesToAggregate.append(data.astype(np.float64))
        actualStartOfFilename = startOfFilename
    
    #A last iteration is needed for the last element
# aggregatedVoices[len(aggregatedVoices)-1]  =  (actualStartOfFilename, aggregatedVoices[len(aggregatedVoices)-1] / numberOfFilesMixed) 
aggregatedVoice = dtw_barycenter.dba(voicesToAggregate,voicesToAggregate[0], use_c=True)
aggregatedVoices.append(aggregatedVoice[:int(shortestLength*samplerate)])

Now we use the gtda code following the tutorial

In [ ]:

# tda magic
from gtda.homology import VietorisRipsPersistence, CubicalPersistence
from gtda.diagrams import PersistenceEntropy, Scaler
from gtda.plotting import plot_heatmap, plot_point_cloud, plot_diagram
from gtda.pipeline import Pipeline
from gtda.time_series import SingleTakensEmbedding, TakensEmbedding
# This does a fixed embedding
# embedding_dimension_periodic = 3
# embedding_time_delay_periodic = 8
# stride = 10

# embedder_periodic = SingleTakensEmbedding(
#     parameters_type="fixed",
#     n_jobs=-1,
#     time_delay=embedding_time_delay_periodic,
#     dimension=embedding_dimension_periodic,
#     stride=stride,
# )
#This looks for the best embedding to represent the time series
embedding_dimension = 7
embedding_time_delay = 23
stride = 1
dimensionsAndTimeDelays = []

for filename, data in aggregatedVoices:
    #Lots of memory is needed with stride 1 and multiple jobs, change with caution

    embedder = SingleTakensEmbedding(
        parameters_type="fixed", n_jobs=8, time_delay=embedding_time_delay, dimension=embedding_dimension, stride=stride
    )
    embedded_signal = embedder.fit_transform(data)
    # # This code saves the embedding, with the first row of the text being the dimension and time_delay used
    # arr0 = [0] * embedder.dimension_
    # arr0[0] = embedder.dimension_
    # arr0[1] = embedder.time_delay_
    # savingArray = np.concatenate(([arr0], embedded_signal))

    savingDirectory = "./embeddedVowels"
    # Create the directory if it doesn't exist
    os.makedirs(savingDirectory, exist_ok=True)

    savingDirectory = savingDirectory + "/" + directory.split('/')[2] 
 
    # Create the directory if it doesn't exist
    os.makedirs(savingDirectory, exist_ok=True)
    savingDirectory = savingDirectory + "/" + directory.split('/')[3] 


    # Create the directory if it doesn't exist
    os.makedirs(savingDirectory, exist_ok=True)

    # Construct the full path for the file
    savingFilename = savingDirectory + "/"+  filename.split('.wav')[0] + 'embedding.csv'
   
    with open(savingFilename, 'wb') as f:
        np.savetxt(f, embedded_signal, delimiter=',')
        print('File ' + savingFilename + ' saved correctly')


File ./embeddedVowels/Control/A/AVPEPUDEAC0001embedding.csvsaved correctly
File ./embeddedVowels/Control/A/AVPEPUDEAC0003embedding.csvsaved correctly
File ./embeddedVowels/Control/A/AVPEPUDEAC0004embedding.csvsaved correctly
File ./embeddedVowels/Control/A/AVPEPUDEAC0005embedding.csvsaved correctly
File ./embeddedVowels/Control/A/AVPEPUDEAC0006embedding.csvsaved correctly
File ./embeddedVowels/Control/A/AVPEPUDEAC0007embedding.csvsaved correctly
File ./embeddedVowels/Control/A/AVPEPUDEAC0008embedding.csvsaved correctly
File ./embeddedVowels/Control/A/AVPEPUDEAC0010embedding.csvsaved correctly
File ./embeddedVowels/Control/A/AVPEPUDEAC0011embedding.csvsaved correctly
File ./embeddedVowels/Control/A/AVPEPUDEAC0012embedding.csvsaved correctly
File ./embeddedVowels/Control/A/AVPEPUDEAC0013embedding.csvsaved correctly
File ./embeddedVowels/Control/A/AVPEPUDEAC0014embedding.csvsaved correctly
File ./embeddedVowels/Control/A/AVPEPUDEAC0015embedding.csvsaved correctly
File ./embeddedVowels/Con

ValueError: too many values to unpack (expected 2)

Persistence Diagram

In [5]:
homology_dimensions = [0, 1, 2]

y_periodic_embedded = y_periodic_embedded[None, :, :]
periodic_persistence = VietorisRipsPersistence(homology_dimensions=homology_dimensions, n_jobs=6)
%time periodic_persistence_diagrams = periodic_persistence.fit_transform(y_periodic_embedded)

CPU times: total: 2.88 s
Wall time: 19min 22s


In [ ]:
plot_diagram(periodic_persistence_diagrams[0])

: 